In [ ]:
from os import listdir, remove
from os.path import isfile, join
from PIL import Image
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split

def read_folders(path, start=0, end=100):
    elems = {}
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if not isfile(file_path):
            print(f'... {file_name} ...')
            elems[file_name] = read_paths_cv2(file_path, start, end)
    return elems

def read_paths_cv2(path, start=0, end=100):
    new_size = (128, 128)
    animals = []
    limit = start
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if limit >= end:
            break
        if not isfile(file_path):
            print(f'WARN - ignoring {file_name}')
            continue

        try:
            img = img = cv2.imread(file_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_resized = cv2.resize(gray, new_size, interpolation=cv2.INTER_AREA)
            if any('' in x for x in gray_resized):
                print(f'ERROR - {file_path} - empty content')
            animals.append(gray_resized)
        except:
            print(f'ERROR - {file_path}')
        limit += 1

    return animals

In [ ]:
print('start reading imgs....')
elems = read_folders('data/img_train/', end=500)
print('end reading imgs')
print('data transformed!')
#cv2.imshow("image", elems[5])
#cv2.waitKey()
#print(elems[5].shape)

In [ ]:
# define a dict with animal - number
animal_names = {
    'butterfly': 0,
    'cat': 1,
    'chicken': 2,
    'cow': 3,
    'dog': 4,
    'elephant': 5,
    'horse': 6,
    'sheep': 7,
    'spider': 8,
    'squirrel': 9
}

In [ ]:
# create an array with all data to get the train - test
full_array = []
targets = []
for k,v in elems.items():
    full_array = full_array + v
    targets = targets + ([animal_names[k]] * len(v))

full_array = np.asarray(full_array)
targets = np.asarray(targets)

In [ ]:
##################
# OPTION 2
# rebase pixels from 0 - 255 to 0 - 1 vals
full_array = []
targets = []
for k,v in elems.items():
    subset = []
    for sublist in v:
        subset.append(sublist / 255)
    full_array = full_array + subset
    targets = targets + ([animal_names[k]] * len(v))

full_array = np.asarray(full_array)
targets = np.asarray(targets)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(full_array, targets, test_size=0.2, random_state=73)

In [ ]:
# create model
# flatten shape
# test 1
model = keras.Sequential([
    Flatten(input_shape=(128, 128)),
    Dense(8192, activation='relu'),
    Dense(2048, kernel_initializer='lecun_normal', activation='selu'),
    Dense(512, kernel_initializer='lecun_normal', activation='selu'),
    Dense(128, kernel_initializer='lecun_normal', activation='selu'),
    Dense(10, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=15)


### TEST 2 --> IGNORING GRAY SCALE AND SEARCH FOR PATTER INSIDE IMG


In [ ]:
#############
# FUNCTIONS WITH COLOR

def read_folders_color(path, start=0, end=100):
    elems = {}
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if not isfile(file_path):
            print(f'... {file_name} ...')
            elems[file_name] = read_paths_cv2_color(file_path, start, end)
    return elems

def read_paths_cv2_color(path, start=0, end=100):
    new_size = (128, 128)
    animals = []
    limit = start
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if limit >= end:
            break
        if not isfile(file_path):
            print(f'WARN - ignoring {file_name}')
            continue

        try:
            img = img = cv2.imread(file_path)
            img_resized = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
            if any('' in x for x in img_resized):
                print(f'ERROR - {file_path} - empty content')
            animals.append(img_resized)
        except:
            print(f'ERROR - {file_path}')
        limit += 1

    return animals

print('start reading imgs....')
elems = read_folders_color('data/img/', end=500)
print('end reading imgs')
print('data transformed!')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(full_array, targets, test_size=0.2, random_state=73)

In [ ]:
# create model
# Conv2D 
# test 2
model = keras.Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    X_train,
    steps_per_epoch=total_train // batch_size,
    epochs=15,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)